In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import csv

DATA_PATH = Path('./data')
HACCP = 'haccp.csv'

pd.set_option('display.max_colwidth', None)

/var/folders/3x/nkdkb4j55gl1v6vcq3yq1yjw0000gn/T/ipykernel_29556/763600736.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


# 01 HACCP

In [2]:
haccp_full_path = DATA_PATH / HACCP
# \n이 포함된 곳: 7760번째 줄의 \n 삭제
haccp_df = pd.read_csv(haccp_full_path, sep='\\', header=None, skip_blank_lines=True) 

print(f'shape: {haccp_df.shape}')
haccp_df.head(1)

shape: (14849, 13)


,0,1,2,3,4,5,6,7,8,9,10,11,12
0,201904962715,설화눈꽃팝김부각스낵,"찹쌀,김,참깨,옥수수기름(옥배유),양파,무,대파,천일염,마늘,새우,멸치,다시마,건표고버섯,둥굴레,감초,정제수",없음,NaN,NaN,남원혼불부각,남원혼불부각,과자,http://fresh.haccp.or.kr/prdimg/2019/201904962715/201904962715-2.jpg,http://fresh.haccp.or.kr/prdimg/2019/201904962715/201904962715-1.jpg,식품,NaN


## 01-1. HACCP 데이터 카테고리 이름 지정
* Response Body 정리
   * \<rnum>1\</rnum> 
   * 품목보고번호: \<prdlstReportNo>201903787251\</prdlstReportNo>
   * 축산/식품구분: \<productGb>식품\</productGb> 
   * 제품명: \<prdlstNm>참군고구마\</prdlstNm>
   * 원재료: \<rawmtrl>군고구마\</rawmtrl>
   * 알레르기유발물질: \<allergy>없음\</allergy>
   * 영양성분: \<nutrient>열량 171kcal,나트륨 8mg,탄수화물 37g,당류 18g,지방 1.4g,트랜스지방 0g,포화지방 0g,콜레스테롤 0g,단백질 2.3g \</nutrient> X
   * 바코드: \<barcode>8 809678 470014\</barcode> 
   * 유형명: \<prdkind>서류가공품\</prdkind> 
   * 유형의상태: \<prdkindstate>반건조\</prdkindstate> 
   * 제조원: \<manufacture>농업회사법인 참살이푸드원(주)\</manufacture> 
   * 판매원: \<seller>\</seller>  X
   * 용량: \<capacity>\</capacity> X
   * 제품이미지URL: \<imgurl1>http://fresh.haccp.or.kr/prdimg/2019/201903787251/201903787251-1.jpg\</imgurl1> ㅇ
   * 메타이미지URL: \<imgurl2>http://fresh.haccp.or.kr/prdimg/2019/201903787251/201903787251-2.jpg\</imgurl2> ㅇ
* matching: **"prdlstReportNo", "prdlstNm", "rawmtr", "allergy", "barcode", "prdkindstate", "manufacture", "seller", "prdkind", "imgurl2", "imgurl1", "productGb", "unknown"**
   * 품목보고번호, 제품명, 원재료, 알레르기유발물지, 바코드, 유형의상태, 제조원, 판매원, 유형명, 메타이미지URL, 제품이미지URL, 축산/식품구분

In [167]:
# 항목 상세확인
haccp_df.loc[8, :]

0                                                              2018048003057
1                                                              홀리닭 통가슴살 오리지널
2                                                                       닭가슴살
3                                                               대두,밀,닭고기,쇠고기
4                                                                        NaN
5                                                                        NaN
6                                                         농업회사법인 유한회사 더원푸드지점
7                                                         농업회사법인 유한회사 더원푸드지점
8                                                                        양념육
9     http://fresh.haccp.or.kr/prdimg/2018/2018048003057/2018048003057-2.jpg
10    http://fresh.haccp.or.kr/prdimg/2018/2018048003057/2018048003057-1.jpg
11                                                                        식품
12                                                                       NaN

In [168]:
# header 이름 재지정
haccp_df.columns = ["prdlstReportNo", "prdlstNm", "rawmtr", "allergy", "barcode", "prdkindstate", "manufacture", "seller", "prdkind", "imgurl2", "imgurl1", "productGb", "unknown"]
haccp_df.head(1)

,prdlstReportNo,prdlstNm,rawmtr,allergy,barcode,prdkindstate,manufacture,seller,prdkind,imgurl2,imgurl1,productGb,unknown
0,201904962715,설화눈꽃팝김부각스낵,"찹쌀,김,참깨,옥수수기름(옥배유),양파,무,대파,천일염,마늘,새우,멸치,다시마,건표고버섯,둥굴레,감초,정제수",없음,NaN,NaN,남원혼불부각,남원혼불부각,과자,http://fresh.haccp.or.kr/prdimg/2019/201904962715/201904962715-2.jpg,http://fresh.haccp.or.kr/prdimg/2019/201904962715/201904962715-1.jpg,식품,NaN


## 01-2. 데이터 정제
1. unknown 열은 값이 전부 `NaN`이므로 삭제
2. 각 열마다 `NaN`값 처리

### 1. unknown 열 drop

In [169]:
haccp_df[haccp_df['unknown'].notna()]

,prdlstReportNo,prdlstNm,rawmtr,allergy,barcode,prdkindstate,manufacture,seller,prdkind,imgurl2,imgurl1,productGb,unknown


In [170]:
haccp_df = haccp_df.drop(columns=['unknown'])
haccp_df.head(1)

,prdlstReportNo,prdlstNm,rawmtr,allergy,barcode,prdkindstate,manufacture,seller,prdkind,imgurl2,imgurl1,productGb
0,201904962715,설화눈꽃팝김부각스낵,"찹쌀,김,참깨,옥수수기름(옥배유),양파,무,대파,천일염,마늘,새우,멸치,다시마,건표고버섯,둥굴레,감초,정제수",없음,NaN,NaN,남원혼불부각,남원혼불부각,과자,http://fresh.haccp.or.kr/prdimg/2019/201904962715/201904962715-2.jpg,http://fresh.haccp.or.kr/prdimg/2019/201904962715/201904962715-1.jpg,식품


### 2. 각 열마다 `NaN`값 처리

In [171]:
# NaN이 존재하는 열 => 전부
haccp_df.columns[haccp_df.isna().any()].tolist()

['prdlstReportNo',
 'prdlstNm',
 'rawmtr',
 'allergy',
 'barcode',
 'prdkindstate',
 'manufacture',
 'seller',
 'prdkind',
 'imgurl2',
 'imgurl1',
 'productGb']

#### prdlstReportNo & prdlstNm
* `prdlstReportNo`가 NaN인 경우는 원재료가 너무 길어서 줄바꿈이 생긴 상황임. 윗 행으로 옮겨주는 방법으로 해결
* `prdlstNm`가 NaN인 경우는 한 상품에 여러 가지의 제품이 들어있는 경우, 행의 시작이 품목번호가 아님

In [180]:
HACCP_NEW = "haccp_new.csv"
haccp_new_path = DATA_PATH / HACCP_NEW 

with open(haccp_new_path , 'w', newline='', encoding='utf-8') as output_csv:
    csv_writer = csv.writer(output_csv)

    with open(haccp_full_path, 'r', newline='', encoding='utf-8') as input_csv:
        csv_reader = csv.reader(input_csv)

        # 이전 행의 내용을 저장할 변수 초기화
        previous_row = None
        for row in csv_reader:
            # 빈 열 삭제
            if not row:
                continue
            # 행의 첫 번째 열이 백슬래시로 시작하는지 확인
            if row[0].startswith('\\'):
                # 백슬래시로 시작하는 경우, 이전 행과 병합하여 새로운 행 생성
                merged_row = [previous_row[0] + row[0][1:]] + row[1:]
                csv_writer.writerow(merged_row)
            elif not str(row[0][0:10]).isdigit():
                merged_row = [previous_row[0] + row[0][1:]] + row[1:]
                csv_writer.writerow(merged_row)
            else:
                # 백슬래시로 시작하지 않는 경우, 그대로 출력
                csv_writer.writerow(row)
                previous_row = row

In [3]:
HACCP = 'haccp_new.csv'
haccp_full_path = DATA_PATH / HACCP

haccp_df = pd.read_csv(haccp_full_path, sep='\\', header=None) 
print(f'shape: {haccp_df.shape}')

haccp_df.columns = ["prdlstReportNo", "prdlstNm", "rawmtr", "allergy", "barcode", "prdkindstate", "manufacture", "seller", "prdkind", "imgurl2", "imgurl1", "productGb", "unknown"]
haccp_df = haccp_df.drop(columns=['unknown'])
haccp_df.head(1)

shape: (14846, 13)


,prdlstReportNo,prdlstNm,rawmtr,allergy,barcode,prdkindstate,manufacture,seller,prdkind,imgurl2,imgurl1,productGb
0,201904962715,설화눈꽃팝김부각스낵,"찹쌀,김,참깨,옥수수기름(옥배유),양파,무,대파,천일염,마늘,새우,멸치,다시마,건표고버섯,둥굴레,감초,정제수",없음,NaN,NaN,남원혼불부각,남원혼불부각,과자,http://fresh.haccp.or.kr/prdimg/2019/201904962715/201904962715-2.jpg,http://fresh.haccp.or.kr/prdimg/2019/201904962715/201904962715-1.jpg,식품


In [4]:
haccp_df.columns[haccp_df.isna().any()].tolist()

['rawmtr',
 'allergy',
 'barcode',
 'prdkindstate',
 'manufacture',
 'seller',
 'prdkind',
 'imgurl2',
 'imgurl1',
 'productGb']

### rawmtr & allergy
* `rawmtr`이 NaN인 경우 같은 품목보고번호인 제품이 있는 경우임
    * 각 항목마다 유용한 정보를 가지고 있는 것이 다르기 때문에 일일이 해줘야 할듯

In [5]:
haccp_df[haccp_df['rawmtr'].isna()]

,prdlstReportNo,prdlstNm,rawmtr,allergy,barcode,prdkindstate,manufacture,seller,prdkind,imgurl2,imgurl1,productGb
2327,2009040500836,당신의부엌한우,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2329,2009040500826,횡성한우진한,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2337,2009040500291,종가집 데이즈,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2638,2008061458767,아워홈 고려삼계탕,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3035,200804050043,유기농 힘쎄지,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3037,2008040500412,유기농 참맛나지,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3475,20070405014119,행복한맛남 프리미엄 무항생제,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3477,20070405014118,행복한맛남 프리미엄 무항생제,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8332,1996051205531,새우표 올리브유로,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10269,19920461131124,부드럽게 씹히는 구수한,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# haccp_df[haccp_df['prdlstReportNo'].duplicated()]
duplicated = set(haccp_df[haccp_df['prdlstReportNo'].duplicated()]['prdlstReportNo'].to_list())
print(f'length: {len(duplicated)}')
s = ''
for pno in duplicated:
    s += pno
    s += ' '
print(s)

length: 132
2004049023115 2004049023126 1990055500385 20070405014118 19900555003157 1989047400843 1995046407072 19860309018759 20080493074106 1983036001676 19900555003130 19940461151561 19900490051429 198506140552563 19940461151655 19940461151335 19910144038103 19970515071392 2009044006841 1989047400899 19860309018765 19800347002343 19880311002411 20030372137301 20030372137320 199005710464 19900555003158 19930618060775 19920444004237 20020614179556 2006037812258 19990386335238 1996051205531 19990144396160 2006039211868 19980568016143 1991037209244 20020614179875 19830360016137 2006037812292 200604731474 20030221351179 20060257026195 1996061431545 19630144002120 19860443015232 19990386335211 2008061458767 20020415029566 19790257001159 2006060794648 200204671276 1997046417285 2004049023111 19860309018500 2003036429887 1994037708480 19900275002276 1999036415949 2011044902380 19960399065288 2007014900257 2004049023138 19860309018501 19900555003180 19870506066269 20060607946120 201403470465

In [7]:
def custom_agg(column):
    if column.name == 'rawmtr':
        combined_string = ', '.join(str(val) for val in column)
        return f'[{combined_string}]'
    else:
        return column.iloc[-1]

haccp_df_unique = haccp_df.groupby('prdlstReportNo').agg(custom_agg).reset_index()
haccp_df_unique[haccp_df_unique['prdlstReportNo'].duplicated()]

,prdlstReportNo,prdlstNm,rawmtr,allergy,barcode,prdkindstate,manufacture,seller,prdkind,imgurl2,imgurl1,productGb


In [10]:
haccp_df_unique[(haccp_df_unique['prdlstReportNo']=="1996060739261")]
haccp_df[(haccp_df['prdlstReportNo']=="1996060739261")]

,prdlstReportNo,prdlstNm,rawmtr,allergy,barcode,prdkindstate,manufacture,seller,prdkind,imgurl2,imgurl1,productGb
8242,1996060739261,표고버섯어묵,"냉동연육(어육살,백설탕,D-소르비톨,산도조절제/수입산),소맥전분(밀/미국,호주산),표고버섯(수입산)5.92%,당근(국산),파(국산),정제소금,L-글루타민산나트륨(향미증진제),글루코노델타락톤,소르빈산칼륨(합성보존료),D-키실로오스,글리신(향미증진제),탄산칼슘(영양강화제),불고기맛엑기스(소스류),대두유(대두)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8243,1996060739261,표고버섯어묵,냉동연육(어육살알수없음,8801793015163,유탕처리제품/비살균제품,(주)환공식품_경남 김해시 진영읍 의전리 280번지,(주)환공식품_경남 김해시 진영읍 의전리 280번지,어묵,http://fresh.haccp.or.kr/prdimg/1996/1996060739261/1996060739261-2.jpg,http://fresh.haccp.or.kr/prdimg/1996/1996060739261/1996060739261-1.jpg,식품,NaN
